In [ ]:
import os
from pathlib import Path

import boto3
from boto3 import client
from botocore.exceptions import ClientError

In [ ]:
def create_bucket(bucket_name, region = None):
    try:
        if region is None:
            s3_client = boto3.client('s3')
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client('s3', region_name = region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket = bucket_name,
                                    CreateBucketConfiguration = location)
    except ClientError as e:
        logging.error(e)
        return False
    return True

def get_buckets(s3_client):
    buckets = []
    response = s3_client.list_buckets()
    for bucket in response['Buckets']:
        buckets.append(bucket["Name"])
    return buckets

def get_file_folders(s3_client, bucket_name, prefix = ''):
    file_names = []
    folders = []

    default_kwargs = {
        'Bucket': bucket_name,
        'Prefix': prefix
    }
    next_token = ''

    while next_token is not None:
        updated_kwargs = default_kwargs.copy()
        if next_token != '':
            updated_kwargs['ContinuationToken'] = next_token

        response = s3_client.list_objects_v2(**default_kwargs)
        contents = response.get('Contents')
        if contents is None:
            return None, None
        for result in contents:
            key = result.get('Key')
            if key[-1] == '/':
                folders.append(key)
            else:
                file_names.append(key)

        next_token = response.get('NextContinuationToken')

    return file_names, folders

def download_files(s3_client, bucket_name, local_path, file_names, folders):

    local_path = Path(local_path)

    for folder in folders:
        folder_path = Path.joinpath(local_path, folder)
        folder_path.mkdir(parents = True, exist_ok = True)

    for file_name in file_names:
        file_path = Path.joinpath(local_path, file_name)
        file_path.parent.mkdir(parents = True, exist_ok = True)
        s3_client.download_file(
            bucket_name,
            file_name,
            str(file_path)
        )
        
def print_buckets(s3_client):
    response = s3_client.list_buckets()

    print('Existing buckets:')
    for bucket in response['Buckets']:
        print(f'  {bucket["Name"]}')
        
def print_files_in_buckets(bucket_name):
    s3 = boto3.resource('s3')
    my_bucket = s3.Bucket(bucket_name)

    for object_summary in my_bucket.objects.filter():
        print(f'-- {object_summary.key}')
    print()

In [ ]:
s3 = boto3.client('s3')
print_buckets(s3)
print()
for bucket in get_buckets(s3):
    print('Bucket Name:', bucket)
    print_files_in_buckets(bucket)

In [33]:
s3 = boto3.client("s3")
path = os.path.join(os.getcwd(), 'bucket/')

buckets = get_buckets(s3)
for bucket in buckets:
    if not os.path.exists(path + bucket + '/'):
        os.mkdir(path + bucket + '/')
    print(bucket)
    file_names, folders = get_file_folders(s3, bucket)
    if file_names is None:
        continue
    download_files(
            s3,
            bucket,
            path + bucket + '/',
            file_names,
            folders
        )